This notebook is an attempt at accomplishing the same goal as the paper* What Can Transformers Learn In-Context? A Case Study of Simple Function Classes*. It doesn't really work because our training and tokenization schema are different. The paper creates a learned linear transformation to convert their vector sequences into vectors of a dimension that GPT-2 can handle, and then they do the same thing to transform GPT-2's output vector into a vector that they are looking for. In addition, they create a new training schema that only learns the output vectors, not every vector in the sequence. 

In [ ]:
!pip install numpy
!pip install transformers
!pip install torch
!pip install sentence-transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.23.1-py3-none-any.whl (5.3 MB)
  Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)
     |████████████████████████████████| 7.6 MB 33.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.2 MB/s 
     |████████████████████████████████| 1.3 MB 46.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=8b1a1914710529095a9b8409cfa44120ce29bf9290eea480d3bdf3e150fb7d41
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built 

In [ ]:
def arr_to_str(in_arr):
  mystr = str(in_arr)
  mystr = mystr[1:len(mystr)-1]
  return mystr

In [ ]:
import random
import numpy as np

'''arr = []
y_arr = []

for i in range(10000):
  tmp_arr = []
  m = random.randrange(-500, 500, 1)
  b = random.randrange(-500, 500, 1)
  for j in range(100):
    x = random.randrange(-500, 500, 1)
    tmp_arr.append(x)
    tmp_arr.append(m*x+b)
  x = random.randrange(-500, 500, 1)
  tmp_arr.append(x)
  y_arr.append(str(m*x+b))
  arr.append(arr_to_str(tmp_arr))'''

arr = []

for i in range(100000):
  tmp_arr = []
  m = random.randrange(-500, 500, 1)
  b = random.randrange(-500, 500, 1)
  for j in range(20):
    x = random.randrange(-500, 500, 1)
    tmp_arr.append(x)
    tmp_arr.append(m*x+b)
  arr.append(arr_to_str(tmp_arr))

In [ ]:
from transformers import AutoTokenizer
import pandas as pd
import datasets

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

def preprocess(in_str):
  return tokenizer(in_str)


tokenized_arr = list(map(preprocess, arr))

train_df = pd.DataFrame(tokenized_arr[:90000])
test_df = pd.DataFrame(tokenized_arr[90000:])

train_ds = datasets.Dataset.from_pandas(train_df)
test_ds = datasets.Dataset.from_pandas(test_df)


Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2").to("cuda")

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    remove_unused_columns=False,
    save_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=data_collator,
)

trainer.train()

Downloading:   0%|          | 0.00/353M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 90000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 33750
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.560900,2.493951
2,2.511500,2.447800


***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8


Epoch,Training Loss,Validation Loss
1,2.560900,2.493951
2,2.511500,2.447800
3,2.492100,2.432637


***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=33750, training_loss=2.5623193178530093, metrics={'train_runtime': 8077.2111, 'train_samples_per_second': 33.427, 'train_steps_per_second': 4.178, 'total_flos': 9070306323922944.0, 'train_loss': 2.5623193178530093, 'epoch': 3.0})

In [7]:
trainer.save_model("./model")

Saving model checkpoint to ./model
Configuration saved in ./model/config.json
Model weights saved in ./model/pytorch_model.bin


In [8]:
from transformers import pipeline, set_seed

model2 = AutoModelForCausalLM.from_pretrained("./model")

in_arr = []
m = 2
b = 1
for j in range(19):
  x = random.randrange(-500, 500, 1)
  in_arr.append(x)
  in_arr.append(m*x+b)
in_arr.append(5)
input = arr_to_str(in_arr)

generator = pipeline('text-generation', model=model2, tokenizer=tokenizer, device=0)
set_seed(42)
generator(input, max_new_tokens = 5, num_return_sequences=5)

loading configuration file ./model/config.json
Model config GPT2Config {
  "_name_or_path": "./model",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "

[{'generated_text': '154, 309, -386, -771, -475, -949, 259, 519, -219, -437, -250, -499, -272, -543, -358, -715, 254, 509, -396, -791, 192, 385, 258, 517, 413, 827, 58, 117, -411, -821, 104, 209, -68, -135, -468, -935, -470, -939, 5, 10, -315'},
 {'generated_text': '154, 309, -386, -771, -475, -949, 259, 519, -219, -437, -250, -499, -272, -543, -358, -715, 254, 509, -396, -791, 192, 385, 258, 517, 413, 827, 58, 117, -411, -821, 104, 209, -68, -135, -468, -935, -470, -939, 5, -10, -'},
 {'generated_text': '154, 309, -386, -771, -475, -949, 259, 519, -219, -437, -250, -499, -272, -543, -358, -715, 254, 509, -396, -791, 192, 385, 258, 517, 413, 827, 58, 117, -411, -821, 104, 209, -68, -135, -468, -935, -470, -939, 5, 48, -261'},
 {'generated_text': '154, 309, -386, -771, -475, -949, 259, 519, -219, -437, -250, -499, -272, -543, -358, -715, 254, 509, -396, -791, 192, 385, 258, 517, 413, 827, 58, 117, -411, -821, 104, 209, -68, -135, -468, -935, -470, -939, 5, -2, -'},
 {'generated_text': '